Film Agent

#### > Setup

In [1]:
!pip freeze | grep boto3

boto3==1.38.13


### Analyze and extract video

In [2]:
import boto3
import json
import uuid
import sagemaker
import uuid

sess = sagemaker.Session()
bucket = sess.default_bucket() # Set a default S3 bucket
region = sess.boto_region_name
prefix = "media-operations-agent-claude"

bda_client = boto3.client('bedrock-data-automation')
bda_runtime_client = boto3.client('bedrock-data-automation-runtime')
s3_client = boto3.client('s3')

#access account id
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]

default_profile_arn = f"arn:aws:bedrock:{region}:{account_id}:data-automation-profile/us.data-automation-v1"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


### Create a BDA project

In [5]:
response = bda_client.create_data_automation_project(
    projectName=f'{prefix}-{str(uuid.uuid4())[0:4]}',
    projectDescription='Media operations agent',
    projectStage='DEVELOPMENT',
    standardOutputConfiguration={
        'video': {
            'extraction': {
                'category': {
                    'state': 'ENABLED',
                    'types': ['TEXT_DETECTION','TRANSCRIPT'],
                },
                'boundingBox': {
                    'state': 'DISABLED',
                }
            },
            'generativeField': {
                'state': 'ENABLED',
                'types': ['VIDEO_SUMMARY','CHAPTER_SUMMARY','IAB'],
            }
        }
    }
)

video_project_arn = response.get("projectArn")
print("BDA video project ARN:", video_project_arn)

BDA video project ARN: arn:aws:bedrock:us-west-2:376678947624:data-automation-project/03efc3434bbc


In [6]:
video_file = "Meridian_summary.mp4"
s3_key = f"{prefix}/{video_file}"

s3_client.upload_file(video_file, bucket, s3_key)
film_video_s3_path = f"s3://{bucket}/{s3_key}"

### Extract analysis from video

In [7]:
response = bda_runtime_client.invoke_data_automation_async(
    inputConfiguration={
        's3Uri': f's3://{bucket}/{s3_key}'
    },
    outputConfiguration={
        's3Uri': f's3://{bucket}/{prefix}/outputs'
    },
    dataAutomationConfiguration={
        'dataAutomationProjectArn': video_project_arn,
        'stage': 'DEVELOPMENT'
    },
    notificationConfiguration={
        'eventBridgeConfiguration': {
            'eventBridgeEnabled': False
        }
    },
    dataAutomationProfileArn=default_profile_arn
)

invocation_arn = response.get("invocationArn")
print("BDA task started:", invocation_arn)

BDA task started: arn:aws:bedrock:us-west-2:376678947624:data-automation-invocation/2a8d1745-698d-46e4-93aa-713647bf6435


### Wait

In [8]:
import time
from IPython.display import clear_output
from datetime import datetime

status, status_response = None, None
while status not in ["Success","ServiceError","ClientError"]:
    status_response = bda_runtime_client.get_data_automation_status(
        invocationArn=invocation_arn
    )
    status = status_response.get("status")
    clear_output(wait=True)
    print(f"{datetime.now().strftime('%H:%M:%S')} : BDA video task: {status}")
    time.sleep(5)

output_config = status_response.get("outputConfiguration",{}).get("s3Uri")
print("Ouput configureation file:", output_config)

12:44:57 : BDA video task: Success
Ouput configureation file: s3://sagemaker-us-west-2-376678947624/media-operations-agent-claude/outputs/2a8d1745-698d-46e4-93aa-713647bf6435/job_metadata.json


### Access the BDA analaysis result

In [9]:
def read_json_on_s3(s3_uri, s3_client):
    # Parse s3 bucket and key from s3 uri
    s3_bucket = s3_uri.split('/')[2]
    s3_key = s3_uri.replace(f's3://{s3_bucket}/','')
    
    # Read BDA output_config file on S3
    response = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
    file_content = response['Body'].read().decode('utf-8')  # Read the content and decode it to a string
    # Convert the content to JSON
    return json.loads(file_content)

In [10]:
config_data = read_json_on_s3(output_config,s3_client)
print(json.dumps(config_data, indent=4))

{
    "job_id": "2a8d1745-698d-46e4-93aa-713647bf6435",
    "job_status": "PROCESSED",
    "semantic_modality": "VIDEO",
    "output_metadata": [
        {
            "asset_id": 0,
            "asset_input_path": {
                "s3_bucket": "sagemaker-us-west-2-376678947624",
                "s3_key": "media-operations-agent-claude/Meridian_summary.mp4"
            },
            "segment_metadata": [
                {
                    "standard_output_path": "s3://sagemaker-us-west-2-376678947624/media-operations-agent-claude/outputs/2a8d1745-698d-46e4-93aa-713647bf6435/0/standard_output/0/result.json"
                }
            ]
        }
    ]
}


In [11]:
from IPython.display import JSON

result_uri = config_data["output_metadata"][0]["segment_metadata"][0]["standard_output_path"]
result_data = read_json_on_s3(result_uri,s3_client)

JSON(result_data)

<IPython.core.display.JSON object>

In [12]:
video_analysis = {
    "visual_summary": result_data["video"]["summary"],
    "audio_transcription":result_data["video"]["transcript"]["representation"]["text"],
    "metadata":result_data["metadata"]
    
}
video_analysis

{'visual_summary': 'In 1950s Los Angeles, three individuals - a schoolteacher, an insurance salesman, and a retired man - vanish near El Matador, a location known for its large rock formation. Initially suspected of mob-related activities or suicides due to their divorced status, the case takes an unexpected turn when a witness reports seeing a man atop the rock behaving strangely before disappearing during a sudden storm. The witness claims to have seen a woman in a white dress with alabaster skin appear in his place, staring upwards.\n\nThe investigation intensifies as Captain Foster is dispatched to El Matador to look into the matter further. However, he finds no sign of Detective Sullivan, leading to a tense standoff with LAPD officers. The disappearance of the three individuals and the mysterious appearance of the woman in white create an atmosphere of intrigue and unease, with the stormy weather adding to the sense of foreboding. As the investigation continues, the true nature of

## Setup Film Agent

In [13]:
import os
import time
import sys

account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    # 'us.amazon.nova-pro-v1:0',
    'us.anthropic.claude-3-5-sonnet-20241022-v2:0',
    'us.anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0'
]

In [14]:
agent_name = f"f-agent-{agent_suffix}"
lambda_name = f"fn-f-agent-{agent_suffix}"

agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{agent_name}'

knowledge_base_name = f'{agent_name}-02'

suffix = f"{region}-{account_id}"

knowledge_base_description = "KB containing information of all the films"
bucket_name = f'{agent_name}-{suffix}'

cast_table = f'cast-table-{agent_suffix}'
cast_pk = 'id'
cast_sk = 'name'
env_args = [cast_table, cast_pk, cast_sk]
bucket_name

'f-agent-us-west-2-376-us-west-2-376678947624'

### Importing helper functions

On following section, we're adding `bedrock_agent_helper.py` and `knowledge_base_helper` on Python path, so the files can be recognized and their functionalities can be invoked.

Now, you're going to import from helper classes `bedrock_agent_helper.py` and `knowledge_base_helper.py`.
 
Those files contain helper classes totally focused on make labs experience smoothly. 

All interactions with Bedrock will be handled by these classes.

Following are methods that you're going to invoke on this lab:

On `agents.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

On `knowledge_bases.py`:
- `create_or_retrieve_knowledge_base`: Create Knowledge Base on Amazon Bedrock if it doesn't exist or get info about previous created.
- `synchronize_data`: Read files on S3, convert text info into vectors and add that information on Vector Database.

In [16]:
import sys
sys.path.insert(0, '..')

from helper.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from helper.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## Create and syncronize Knowledge Base

On this section, you're going to create a Amazon Bedrock Knowledge Base and ingest data on it.

This data contains basic information about how forecast process is done.

**This creation process can take several minutes.**

In [17]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

Creating KB f-agent-us-west-2-376-02
Step 1 - Creating or retrieving f-agent-us-west-2-376-us-west-2-376678947624 S3 bucket for Knowledge Base documents
Creating bucket f-agent-us-west-2-376-us-west-2-376678947624
Step 2 - Creating Knowledge Base Execution Role (AmazonBedrockExecutionRoleForKnowledgeBase_636) and Policies
Step 3 - Creating OSS encryption, network and data access policies
Step 4 - Creating OSS Collection (this step takes a couple of minutes to complete)
{ 'ResponseMetadata': { 'HTTPHeaders': { 'connection': 'keep-alive',
                                         'content-length': '320',
                                         'content-type': 'application/x-amz-json-1.0',
                                         'date': 'Sun, 11 May 2025 12:47:08 '
                                                 'GMT',
                                         'x-amzn-requestid': 'ddf7fe41-9987-4293-ae22-ecbc3696d447'},
                        'HTTPStatusCode': 200,
                     

### Upload Film Documents to S3

In [18]:
!rm -rf `find -type d -name .ipynb_checkpoints`

In [19]:
films_folder = "films"

!aws s3 sync {films_folder} s3://{bucket_name}/

upload: films/meridian.txt to s3://f-agent-us-west-2-376-us-west-2-376678947624/meridian.txt
upload: films/meridian.txt.metadata.json to s3://f-agent-us-west-2-376-us-west-2-376678947624/meridian.txt.metadata.json


In [20]:
# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

{ 'dataSourceId': 'Q6COH7RAHA',
  'ingestionJobId': 'X80MXUUKFF',
  'knowledgeBaseId': '0VCHHJMGFA',
  'startedAt': datetime.datetime(2025, 5, 11, 12, 50, 47, 980609, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 0,
                  'numberOfMetadataDocumentsModified': 0,
                  'numberOfMetadataDocumentsScanned': 0,
                  'numberOfModifiedDocumentsIndexed': 0,
                  'numberOfNewDocumentsIndexed': 0},
  'status': 'STARTING',
  'updatedAt': datetime.datetime(2025, 5, 11, 12, 50, 47, 980609, tzinfo=tzlocal())}
{ 'dataSourceId': 'Q6COH7RAHA',
  'ingestionJobId': 'X80MXUUKFF',
  'knowledgeBaseId': '0VCHHJMGFA',
  'startedAt': datetime.datetime(2025, 5, 11, 12, 50, 47, 980609, tzinfo=tzlocal()),
  'statistics': { 'numberOfDocumentsDeleted': 0,
                  'numberOfDocumentsFailed': 0,
                  'numberOfDocumentsScanned': 1,
  

In [21]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [22]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """Use this knowledge base when you need to look up title, director, and plot of a film"""
}

## Creating Agent

In [23]:
agent_description = """You are a film analyst. You job is analyzing shorts or derivatives of 
films, like promo or trailer. Finding the matching film and identify key cast memebers."""

agent_instruction = """
Your task is to analyze film videos or deriative of film videos and match it to the correct
film title and identify key cast memebers.

Your capabilities include:
<capabilities>
- Matching video to correct film title
- Identify key cast members
</capabilities>

Response style:
<Response style>
- Be helpful and solution-oriented
- Use clear, non-technical language
- Maintain natural conversation flow
- Be concise yet informative
- do not add extra information not required by the user
</Response style>
"""

film_agent = agents.create_agent(
    agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=False
)
time.sleep(20)
film_agent

('1JQBILFDE7',
 'TSTALIASID',
 'arn:aws:bedrock:us-west-2:376678947624:agent-alias/1JQBILFDE7/TSTALIASID')

### Associating knowledge base
Now that we've created the agent, let's associate the previously created knowledge base to it.

In [24]:
agents.associate_kb_with_agent(
    film_agent[0],
    kb_config['kb_instruction'],
    kb_config['kb_id']
)
time.sleep(20)

## Test Knowledge Base W/ Agent

In [25]:
session_state = {
    'knowledgeBaseConfigurations': [
            {
                'knowledgeBaseId': kb_config['kb_id'],
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'implicitFilterConfiguration': {
                            'metadataAttributes': [
                                {
                                    'description': 'this is the name of the director',
                                    'key': 'Director',
                                    'type': 'STRING'
                                },
                            ],
                            'modelArn': "anthropic.claude-3-5-sonnet-20241022-v2:0"
                        },
                        'numberOfResults': 1,
                        'overrideSearchType': 'HYBRID'
                    }
                }
            },
        ]
}

In [26]:
%%time
import uuid

response = agents.invoke(
    input_text=f"which film is directored by Curtis Clark", 
    agent_id=film_agent[0], 
    enable_trace=True,
    session_id=str(uuid.uuid4()),
    session_state=session_state
)
print("====================")
print(response)

invokeAgent API request ID: 2713edb7-b32a-4a3a-a4b4-2f0ca661ec05
invokeAgent API session ID: da3d87cc-3105-43de-888f-b2f36e0a3329
  agent id: 1JQBILFDE7, agent alias id: TSTALIASID
Using knowledge base id: 0VCHHJMGFA to search for:
  which film is directored by Curtis Clark

Knowledge base lookup output, 1 references:

  (1) Title: Meridian (Short 2016)

Storyline: In 1947 Los Angeles, veteran detective Mac Foster talks with his younger partner Jake Sullivan about the disappearance of three men near a rock overlooking a beach. Mac explains that the three had no connection to one another, except that they were all divorc...

---- Step 1 ----
Took 3.5s, using 771 tokens (in: 708, out: 63) to complete prior action, observe, orchestrate.
Final response:
Curtis Clark directed the 2016 short film "Meridian."...
got 1 citations 

Agent made a total of 1 LLM calls, using 771 tokens (in: 708, out: 63), and took 3.6 total seconds
Curtis Clark directed the 2016 short film "Meridian." [s3://f-agen

In [29]:
%store knowledge_base_name
%store kb_config
%store agent_instruction
%store agent_name

Stored 'knowledge_base_name' (str)
Stored 'kb_config' (dict)
Stored 'agent_instruction' (str)
Stored 'agent_name' (str)


### Creating Lambda

In [30]:
%%writefile detection.py
import boto3
import json
import os
import time
from boto3.dynamodb.conditions import Key, Attr

## DynamoDB parameters
dynamodb_resource = boto3.resource('dynamodb')
cast_table = os.getenv('cast_table')
cast_pk = os.getenv('cast_pk')

## Rekognition parameters
rek_client = boto3.client('rekognition')

def get_named_parameter(event, name):
    try:
        return next(item for item in event['parameters'] if item['name'] == name)['value']
    except StopIteration:
        raise ValueError(f"Required parameter '{name}' not found in event")
        
def get_cast_member(cast_id):
    try:
        table = dynamodb_resource.Table(cast_table)
        key_expression = Key(cast_pk).eq(cast_id)
        query_data = table.query(
                KeyConditionExpression=key_expression
            )
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {cast_table}.')

def start_celebrity_detection(bucket, video_key):
    response = rek_client.start_celebrity_recognition(
        Video={
            'S3Object': {
                'Bucket': bucket,
                'Name': video_key
            }
        }
    )
    return response['JobId']

def get_celebrity_detection_results(job_id):
    response = rek_client.get_celebrity_recognition(JobId=job_id)
    return response

def extract_bucket_key(video_s3_path):
    path = video_s3_path[5:]  # Remove 's3://'
    bucket, key = path.split('/', 1)  # Split into bucket and key   
    return bucket, key

def detect_key_figures(video_s3_path):
    bucket, key = extract_bucket_key(video_s3_path)
    
    job_id = start_celebrity_detection(bucket, key)
    print(f"Started celebrity detection job: {job_id}")

    while True:
        response = get_celebrity_detection_results(job_id)
        status = response['JobStatus']
        
        if status in ['SUCCEEDED', 'FAILED']:
            print("JOB COMPLETE....")
            break
        
        print("Job in progress...")
        time.sleep(10)

    unique_celebrities = {}  # Dictionary to store unique celebrities

    if status == 'SUCCEEDED':
        for celebrity in response['Celebrities']:
            celeb = celebrity['Celebrity']
            
            # Only process celebrities with 95%+ confidence
            if celeb['Confidence'] >= 95.0:
                celeb_id = celeb['Id']
                
                # Store or update celebrity info only if not already stored
                if celeb_id not in unique_celebrities:
                    celebrity_info = {
                        'name': celeb['Name'],
                        'confidence': celeb['Confidence'],
                        'id': celeb_id,
                        'first_appearance': celebrity['Timestamp']
                    }
                    
                    # If you have additional celebrity info in DynamoDB
                    try:
                        query_items = get_cast_member(celeb_id)
                        if query_items:
                            celebrity_info.update(query_items[0])
                    except Exception as e:
                        print(f"Error fetching additional celebrity info: {str(e)}")
                    
                    unique_celebrities[celeb_id] = celebrity_info
    else:
        print("Detection failed....")

    # Convert the dictionary values to a list for the final output
    final_output = list(unique_celebrities.values())
    return final_output

def populate_function_response(event, response_body):
    return {
        'response': {
            'actionGroup': event['actionGroup'],
            'function': event['function'],
            'functionResponse': {
                'responseBody': {
                    'TEXT': {
                        'body': str(response_body)
                    }
                }
            }
        }
    }
    
def lambda_handler(event, context):
    print(event)
    
    function = event.get('function', '')
    parameters = event.get('parameters', [])
    video_s3_path = get_named_parameter(event, "video_s3_path")

    if function == 'detect_key_figures':
        result = detect_key_figures(video_s3_path)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

Overwriting detection.py


In [31]:
functions_def = [
    {
        "name": "detect_key_figures",
        "description": """Detect key figures (celebrity, cast member) from a video
        and retrieve information about their position and team""",
        "parameters": {
            "video_s3_path": {
                "description": "S3 location of the video (e.g: s3://......)",
                "required": True,
                "type": "string"
            }
        }
    }
]

### Creating action group and attaching to the agent
Now it's time to add this Lambda function and the function details as an action group for this agent and prepare it.

In [32]:
agents.add_lambda_action_group_with_rek(
    agent_name=agent_name,
    lambda_function_name=lambda_name,
    source_code_file="detection.py",
    agent_functions=functions_def,
    agent_action_group_name="key_figure_detection_actions",
    agent_action_group_description="Functions to identify key figures and look up cast members and their role from the video",
    env_args=env_args
)
time.sleep(30)

### > Loading dynamoDB

In [33]:
# Open and read the JSONL file
with open('cast_members.jsonl', 'r') as file:
    table_items = [json.loads(line.strip()) for line in file]

agents.load_dynamodb(cast_table, table_items)

In [34]:
resp = agents.query_dynamodb(
    cast_table, cast_pk, '4kn3Xu8r'
)
resp

[{'film': 'Meridian',
  'id': '4kn3Xu8r',
  'role': 'Mac Foster',
  'name': 'Kevin Kilner,'}]

In [36]:
%store cast_table
%store cast_pk
%store film_video_s3_path

Stored 'cast_table' (str)
Stored 'cast_pk' (str)
Stored 'film_video_s3_path' (str)


### Test Agent

In [28]:
session_state = {
    'promptSessionAttributes': {
        "<video_analysis>": json.dumps(video_analysis),
        "<video_s3_path>": film_video_s3_path,
    },
    'knowledgeBaseConfigurations': [
            {
                'knowledgeBaseId': kb_config['kb_id'],
                'retrievalConfiguration': {
                    'vectorSearchConfiguration': {
                        'implicitFilterConfiguration': {
                            'metadataAttributes': [
                                {
                                    'description': 'this is the name of the director',
                                    'key': 'Director',
                                    'type': 'STRING'
                                },
                            ],
                            'modelArn': "anthropic.claude-3-5-sonnet-20241022-v2:0"
                        },
                        'numberOfResults': 1,
                        'overrideSearchType': 'HYBRID'
                    }
                }
            },
        ]
}

In [29]:
%%time
response = agents.invoke(
    input_text=f"""
    Here is a clip extraction:
    $prompt_session_attributes.video_analysis$

    if needed, here is the s3 location of the video:
    $prompt_session_attributes.video_s3_path$
    
    can you tell me which film is this clip from?
    """, 
    agent_id=film_agent[0],
    enable_trace=True,
    session_id=str(uuid.uuid4()),
    session_state=session_state
)
print("====================")
print(response)

invokeAgent API request ID: 4c96b614-3e02-4843-905e-c1b62665820b
invokeAgent API session ID: 94f20e87-24de-43fe-b3a7-c6020e421e3f
  agent id: YMRXSPKGV0, agent alias id: TSTALIASID
---- Step 1 ----
Took 6.2s, using 2109 tokens (in: 1937, out: 172) to complete prior action, observe, orchestrate.
Let me analyze this request. I have a video clip and its analysis that appears to be about mysterious disappearances near El Matador in 1950s Los Angeles. I should:
1. Search the knowledge base to identify the film based on these plot details
2. Verify by detecting key figures from the video
Using knowledge base id: VBY6Z2TPPF to search for:
  movie about disappearances near El Matador rock formation 1950s Los Angeles woman in white dress

Knowledge base lookup output, 1 references:

  (1) Title: Meridian (Short 2016)

Storyline: In 1947 Los Angeles, veteran detective Mac Foster talks with his younger partner Jake Sullivan about the disappearance of three men near a rock overlooking a beach. Mac

### Create alias

As you can see, you can use your agent with the `TSTALIASID` to complete tasks. 
However, for multi-agents collaboration it is expected that you first test your agent and only use it once it is fully functional. 
Therefore to use an agent as a sub-agent in a multi-agent collaboration you first need to create an agent alias and connect it to a new version. 

Since we've tested and validated our agent, let's now create an alias for it:

In [30]:
film_agent_alias_id, film_agent_alias_arn = agents.create_agent_alias(
    film_agent[0], 'v1'
)

### Saving Information

In [31]:
film_agent_arn = agents.get_agent_arn_by_name(agent_name)
film_agent_id = film_agent[0]
film_kb = knowledge_base_name
film_agent_name = agent_name
film_kb_id = kb_config['kb_id']
film_kb_arn = kb_arn
film_kb_description = knowledge_base_description
film_kb_name = knowledge_base_name
film_video_analysis = video_analysis

%store film_agent_alias_id
%store film_agent_alias_arn
%store film_agent_arn
%store film_agent_id
%store film_kb
%store film_agent_name
%store film_kb_id
%store film_kb_arn
%store film_kb_description
%store film_kb_name

%store film_video_analysis
%store film_video_s3_path

Stored 'film_agent_alias_id' (str)
Stored 'film_agent_alias_arn' (str)
Stored 'film_agent_arn' (str)
Stored 'film_agent_id' (str)
Stored 'film_kb' (str)
Stored 'film_agent_name' (str)
Stored 'film_kb_id' (str)
Stored 'film_kb_arn' (str)
Stored 'film_kb_description' (str)
Stored 'film_kb_name' (str)
Stored 'film_video_analysis' (dict)
Stored 'film_video_s3_path' (str)


### Clean Up

In [ ]:
# agents.delete_agent(agent_name)

In [ ]:
# kb.delete_kb(knowledge_base_name)

In [ ]:
# agents.delete_lambda(lambda_function_name=lambda_name)

In [ ]:
# #delete BDA project
# response = bda_client.delete_data_automation_project(
#     projectArn=video_project_arn
# )